<a href="https://colab.research.google.com/github/mjalalimanesh/applied-data-analytics-with-python/blob/main/notebooks/quick_ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Quick Ratio** is an indicator of user grwoth that can combine growth, retention and churn into one number that describes how efficiently your product is growing.

Here is the formula:

\begin{align}
Quick Ratio = \frac{new + ressurected}{churned}
\end{align}

Time interval to compute the numbers in depands on usage frequency of the product. For example Monthly, Quarterly, or Annual.

If a Quick Ratio is >1 the number of users is growing; conversely, if it is <1 the number of users is declining. 

More information can be found in [this](https://medium.com/theventurecity/quick-ratio-as-a-shortcut-to-understand-product-growth-ae60212bd371) blog post by Ekaterina Skorobogatova

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()


In [2]:
url = 'https://raw.githubusercontent.com/mjalalimanesh/applied-data-analytics-with-python/main/database/Superstore.csv?token=GHSAT0AAAAAABZ3QDJUJBLE5GERFVE5TCIKY2G4EQQ'
df = pd.read_csv(url, encoding='unicode_escape')
def tweak_superstore(df):
  return (df
          .rename(columns=lambda c: c.lower().replace(' ', '_').replace('-', '_'))
          .rename(columns={'country/region': 'country'})
          .drop(labels=['ship_date', 'ship_mode', 'postal_code', 'product_id', 'segment', 'city', 'state', 'country', 'row_id', 'sub_category'], axis='columns')
          .assign(order_date= lambda _df: pd.to_datetime(_df.order_date))
          .astype({k: 'category' for k in ['category', 'region']})
  )

orders = tweak_superstore(df)
orders.head(3)

,order_id,order_date,customer_id,customer_name,region,category,product_name,sales,quantity,discount,profit
0,CA-2018-152156,2018-11-08,CG-12520,Claire Gute,South,Furniture,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,CA-2018-152156,2018-11-08,CG-12520,Claire Gute,South,Furniture,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,CA-2018-138688,2018-06-12,DV-13045,Darrin Van Huff,West,Office Supplies,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


In [3]:
quick_ratio_table = \
(orders
 .groupby(by=['customer_id', pd.Grouper(key="order_date", freq="MS")])
 .size()
 .reset_index()
 .drop(labels=[0], axis='columns')
 .assign(rnk = lambda _df: _df
                              .groupby('customer_id')
                              ['order_date']
                              .transform(lambda x: x.rank(method='dense')).astype('int'))
 .assign(
         last_month_purchased=lambda _df: ((_df.order_date - _df.order_date.shift(1) < '32 days'))
         ,user_type=lambda _df: np.select([_df.rnk == 1, (_df.rnk != 1) & _df.last_month_purchased, (_df.rnk != 1) & ~(_df.last_month_purchased)],
                                         ['new', 'retained', 'ressurected']
                                         ))
 .groupby(by=['order_date', 'user_type'])
 ['customer_id']
 .nunique()
 .unstack('user_type')
 .fillna(0)
 .assign(total=lambda _df: _df['new'].add(_df['retained'], fill_value=0).add(_df['ressurected'], fill_value=0),
         churned=lambda _df: _df['total'].shift(1) - _df['retained'],
         quick_ratio=lambda _df: np.round((_df['new'].add(_df['ressurected'], fill_value=0))/_df['churned'], 1))
 )

quick_ratio_table

user_type,new,ressurected,retained,total,churned,quick_ratio
order_date,,,,,,
2016-01-01,32.0,0.0,0.0,32.0,NaN,NaN
2016-02-01,24.0,0.0,3.0,27.0,29.0,0.8
2016-03-01,65.0,0.0,4.0,69.0,23.0,2.8
2016-04-01,56.0,4.0,4.0,64.0,65.0,0.9
2016-05-01,56.0,5.0,6.0,67.0,58.0,1.1
2016-06-01,48.0,10.0,5.0,63.0,62.0,0.9
2016-07-01,44.0,17.0,4.0,65.0,59.0,1.0
2016-08-01,49.0,14.0,7.0,70.0,58.0,1.1
2016-09-01,68.0,40.0,10.0,118.0,60.0,1.8
